In [8]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import metrics
import requests, os
from zipfile import ZipFile
import shutil
from sklearn.cluster import MiniBatchKMeans 
import copy
from sklearn import svm as SVM

Consts

In [9]:
ZIP_PATH = "http://people.csail.mit.edu/torralba/code/spatialenvelope/spatial_envelope_256x256_static_8outdoorcategories.zip"

Helper Functions

In [10]:
'''
Download and unzip file
'''
def load_files():
    filename = ZIP_PATH.split("/")[-1]
    with open(filename, "wb") as f:
        r = requests.get(ZIP_PATH)
        f.write(r.content)
    foldername = filename.split(".")[0]
    with ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()
        nameList = zip_ref.namelist()
        imgs = [name.split("/")[-1] for name in nameList if name.endswith('.jpg')]
        forest_images = [name for name in imgs if name.startswith('forest_')]
        mountain_images = [name for name in imgs if name.startswith('mountain_')]
    shutil.move(foldername, 'images')
    return forest_images,mountain_images


In [11]:
def prepare_set(image_names):
    num_imgs = len(image_names)
    split_index = int(num_imgs*0.8)
    train_set = image_names[:split_index-1]
    test_set = image_names[split_index:]
    return train_set, test_set

In [12]:
images_set1,images_set2 = load_files()
train_set_1, test_set_1 = prepare_set(images_set1)
train_set_2, test_set_2 = prepare_set(images_set2)
train_set = train_set_1+train_set_2
test_set = test_set_1+test_set_2

In [13]:
def get_image_by_name(filename):
    filename = 'images//' + filename
    image = cv2.imread(filename)
    return image


1. Compute dense SIFT from the images

In [14]:
def get_danse_SIFT(filename):
    image = get_image_by_name(filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    key_point = [cv2.KeyPoint(x, y, 8) for x in range(0, gray_image.shape[0], 8) for y in range(0, gray_image.shape[1], 8)]
    dense_shift = sift.compute(image, key_point)[1]
    return dense_shift 

In [15]:
def get_dense_sift_set(image_set):
    set_x = []
    set_y = []
    for filename in image_set:
        res = get_danse_SIFT(filename)
        set_x.append(res)
        set_y.append(filename)     
    return set_x, set_y

In [16]:
x_train, y_train = get_dense_sift_set(train_set)
x_test, y_test = get_dense_sift_set(test_set)

In [17]:
print(len(x_train))
print(len(y_train))

559
559


In [18]:
#for c in train_x:
#    denseSift = getDenseSift(c)
#    denseSiftLength = len(denseSift)
#    for index in range(denseSiftLength):
#        lines[int(numOfLines * cCounter + index)] = denseSift[index]
#    cCounter += 1
#kmeans = MiniBatchKMeans(n_clusters=2, random_state=0, batch_size=6, max_iter=10)
#kmeans.fit(lines)
#get_sets_x_y(train_set, kmeans)

In [19]:
def get_all_hists(x_train, y_train):
    print(len(x_train))
    print(len(y_train))
    all_train_x = []
    all_train_y = []
    for i in range(len(x_train)):
        for j in range(x_train[i].shape[0]):
            all_train_x.append(x_train[i][j,:])
            all_train_y.append(y_train[i])
    return np.array(all_train_x),np.array(all_train_y)

In [20]:
def calculate_kmeans(all_train_desc, k):
    kmeans = MiniBatchKMeans (n_clusters=k)
    kmeans.fit(all_train_desc)
    return kmeans

In [21]:
# form training set histograms for each training image using BoW representation
def calculate_histogram(x_train, kmeans, k):
    hist_set = []
    set_y = []
    for elm in x_train:
        predict = kmeans.predict(elm)
        hist_set.append(np.bincount(predict, minlength=k).reshape(1,-1).ravel())
        set_y.append(elm)     
    return np.array(hist_set), np.array(set_y)

In [22]:
k = 100
all_hists_x, all_hists_y = get_all_hists(x_train, y_train)
kmeans = calculate_kmeans(all_hists_x, k)


559
559


In [23]:
train_hist, train_y  = calculate_histogram(x_train, kmeans, k)
test_hist, test_y = calculate_histogram(x_test, kmeans, k)

In [24]:
def train_svm(X, y):
    # Train an SVM classifier
    svm = SVM.SVC(kernel="linear", cache_size=4000)
    svm.fit(X, y)    
    return svm

In [25]:
print(train_hist.shape)
print(train_y.shape)
svm = train_svm(all_hists_x,all_hists_y)


(559, 100)
(559, 1024, 128)


In [ ]:
#for image in test_hist:
predict = svm.predict(train_y)
accuracyScore = metrics.accuracy_score(predict, test_y) * 100
print("the Prediction Accuracy Score is: " + str(accuracyScore) + "%, C is: " +image)

In [ ]:
#os.rmdir(foldername)